# Analyze results


In [7]:
from pathlib import Path

import numpy as np
import pandas as pd
import sklearn.metrics as M
from scipy.special import expit
from tqdm.notebook import tqdm

In [8]:
results_root = Path('../results/')
results_model_folder = list(results_root.glob('net-F3Net_traindb-celebdf_face-scale_size-224_seed-41_bestval'))

In [9]:
def compute_metrics(df_res:pd.DataFrame,train_tag:str) -> dict:
    numreal = sum(df_res['label']==False)
    numfake = sum(df_res['label']==True
)
    
    netname = train_tag.split('net-')[1].split('_')[0]
    traindb = train_tag.split('traindb-')[1].split('_')[0]
    
    loss = M.log_loss(df_res['label'],expit(df_res['score']))
    acc = M.accuracy_score(df_res['label'],df_res['score']>0)
    accbal = M.balanced_accuracy_score(df_res['label'],df_res['score']>0)
    rocauc = M.roc_auc_score(df_res['label'],df_res['score'])
    
    res_dict = {'traintag':train_tag,
                'net':netname,
                'traindb': traindb,
                'testdb':testdb,'testsplit':testsplit,
                'numreal':numreal,'numfake':numfake,
                'loss':loss,
                'acc':acc,'accbal':accbal,
                'rocauc':rocauc} 
    return res_dict

In [10]:
results_frame_list = []
results_video_list = []

for model_folder in tqdm(results_model_folder):
    train_model_tag = model_folder.name
    model_results = model_folder.glob('*.pkl')
    for model_path in model_results:
        testdb,testsplit = model_path.with_suffix('').name.rsplit('_',1)
        
        df_frames = pd.read_pickle(model_path)
        results_frame_list.append(compute_metrics(df_frames,train_model_tag))
        
        df_videos = df_frames[['video','label','score']].groupby('video').mean()
        df_videos['label'] = df_videos['label'].astype(np.bool)
        results_video_list.append(compute_metrics(df_videos,train_model_tag))

  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\eun\AppData\Local\Temp\ipykernel_22684\2771310375.py:14: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df_videos['label'] = df_videos['label'].astype(np.bool)


In [11]:
df_res_frames = pd.DataFrame(results_frame_list)
df_res_frames

,traintag,net,traindb,testdb,testsplit,numreal,numfake,loss,acc,accbal,rocauc
0,net-F3Net_traindb-celebdf_face-scale_size-224_...,F3Net,celebdf,celebdf,val,9226,63312,0.149578,0.948468,0.955526,0.989835


In [12]:
df_res_video = pd.DataFrame(results_video_list)
df_res_video

,traintag,net,traindb,testdb,testsplit,numreal,numfake,loss,acc,accbal,rocauc
0,net-F3Net_traindb-celebdf_face-scale_size-224_...,F3Net,celebdf,celebdf,val,290,1979,0.043213,0.984134,0.985019,0.998643


In [7]:
df_res_frames.to_csv(results_root.joinpath('frames.csv'),index=False)
df_res_video.to_csv(results_root.joinpath('videos.csv'),index=False)
